# Planning Team Review Meeting - Post-Execution Cycle

**Purpose:** Review Executive Team execution results and plan next cycle

**Flow:**
1. 📥 Read execution results from Executive Team
2. 🤖 Initialize Planning Team (4 agents)
3. 📊 Analyze results and assess progress
4. 🎯 Generate new `pending_actions.json` for next cycle
5. 💾 Save to Google Drive

**Timeline:** After each Executive Team execution cycle

---
## Setup: Mount Google Drive & Load API Keys

In [ ]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

import os
import sys
from pathlib import Path

# Set project paths
GDRIVE_ROOT = '/content/drive/MyDrive/cv_multimodal/project'
PROJECT_ROOT = f'{GDRIVE_ROOT}/computer-vision-clean'
MULTI_AGENT_ROOT = f'{PROJECT_ROOT}/multi-agent'

# Add to Python path
sys.path.insert(0, MULTI_AGENT_ROOT)
sys.path.insert(0, PROJECT_ROOT)

print(f"✅ Google Drive mounted")
print(f"📁 Project root: {PROJECT_ROOT}")
print(f"🤖 Multi-agent root: {MULTI_AGENT_ROOT}")

In [ ]:
import os
from pathlib import Path

print("="*80)
print("🔍 FINDING AND LOADING API KEYS")
print("="*80)

# Search for .env file in multiple locations
search_paths = [
    f'{GDRIVE_ROOT}/.env',
    f'{GDRIVE_ROOT}/computer-vision-clean/.env',
    '/content/drive/MyDrive/cv_multimodal/.env',
    '/content/drive/My Drive/cv_multimodal/project/.env',
    f'{PROJECT_ROOT}/.env'
]

env_file = None
for path in search_paths:
    if Path(path).exists():
        env_file = Path(path)
        print(f"\n✅ Found .env file: {path}")
        print(f"   Size: {env_file.stat().st_size} bytes\n")
        break
    else:
        print(f"   Checking: {path}... not found")

if not env_file:
    print("\n🔍 Searching entire cv_multimodal directory...")
    base = Path('/content/drive/MyDrive/cv_multimodal')
    if base.exists():
        all_env = list(base.rglob('*.env')) + list(base.rglob('.env*'))
        if all_env:
            print(f"\n✅ Found .env files:")
            for f in all_env:
                print(f"   📄 {f}")
            env_file = all_env[0]
    
    if not env_file:
        print("\n❌ No .env file found!")
        raise FileNotFoundError("No .env file found - check Google Drive")

# Load all API keys
loaded_keys = []
with open(env_file, 'r') as f:
    for line in f:
        line = line.strip()
        if line and not line.startswith('#') and '=' in line:
            key, value = line.split('=', 1)
            key = key.strip()
            value = value.strip().strip('"').strip("'")  # Remove quotes
            os.environ[key] = value
            loaded_keys.append(key)

# Verify required keys for Planning Team
required = {
    'ANTHROPIC_API_KEY': 'Claude Opus 4 (Strategic Leader), Sonnet 4 (Empirical Lead)',
    'OPENAI_API_KEY': 'GPT-4 (Critical Evaluator)',
    'GOOGLE_API_KEY': 'Gemini (Research Advisor)'
}

print("Verifying required API keys:\n")
all_loaded = True

for key, usage in required.items():
    value = os.environ.get(key)
    if value and len(value) > 10:
        masked = f"{value[:8]}...{value[-4:]}"
        print(f"✅ {key}")
        print(f"   Value: {masked}")
        print(f"   Used by: {usage}\n")
    else:
        print(f"❌ {key} - NOT FOUND OR INVALID")
        print(f"   Needed by: {usage}\n")
        all_loaded = False

print("="*80)
if all_loaded:
    print("✅ ALL API KEYS LOADED SUCCESSFULLY")
    print(f"✅ Loaded {len(loaded_keys)} total keys from: {env_file}")
    print("✅ Ready to initialize Planning Team")
else:
    print("❌ SOME API KEYS MISSING OR INVALID")
    print("⚠️ Planning Team execution will fail without all 3 keys")
    raise ValueError("Missing required API keys")
print("="*80)

In [ ]:
# Install dependencies
!pip install -q anthropic openai google-generativeai python-dotenv pyyaml tiktoken

print("✅ Dependencies installed")

---
## Phase 1: Read Execution Results from Executive Team

In [ ]:
import json
from datetime import datetime
from pathlib import Path

print("="*80)
print("📥 READING EXECUTIVE TEAM EXECUTION RESULTS")
print("="*80)

# Read progress update (main handoff file)
progress_file = Path(MULTI_AGENT_ROOT) / 'reports/handoff/execution_progress_update.md'

if not progress_file.exists():
    print(f"❌ No execution results found at {progress_file}")
    print("⚠️ Executive Team must complete execution first")
    raise FileNotFoundError(f"Missing: {progress_file}")

with open(progress_file, 'r') as f:
    progress_text = f.read()

print(f"\n✅ Found execution progress report")
print(f"   File: {progress_file}")
print(f"   Size: {len(progress_text)} characters\n")

# Find most recent JSON results
results_dir = Path(MULTI_AGENT_ROOT) / 'reports/execution/results'
json_files = list(results_dir.glob('execution_results_*.json'))

if json_files:
    # Get most recent by modification time
    latest_json = max(json_files, key=lambda p: p.stat().st_mtime)
    with open(latest_json, 'r') as f:
        results_data = json.load(f)
    
    print(f"✅ Found execution results JSON")
    print(f"   File: {latest_json.name}")
    print(f"   Tasks: {results_data.get('total_tasks', 'N/A')}")
    print(f"   Completed: {results_data.get('completed', 'N/A')} ✅")
    print(f"   Failed: {results_data.get('failed', 'N/A')} ❌")
else:
    results_data = {}
    latest_json = None
    print("⚠️ No JSON results found, using markdown only")

print("\n" + "="*80)
print("📊 EXECUTION SUMMARY FOR PLANNING TEAM REVIEW")
print("="*80)

if results_data:
    print(f"\n📋 Executive Team Results:")
    print(f"   Total tasks: {results_data.get('total_tasks', 'N/A')}")
    print(f"   Completed: {results_data.get('completed', 'N/A')} ✅")
    print(f"   Failed: {results_data.get('failed', 'N/A')} ❌")
    print(f"   Duration: {results_data.get('total_duration_seconds', 0):.1f}s")

print(f"\n📄 Progress Report Preview:")
print(progress_text[:500] + "...")

print("\n" + "="*80)

---
## Phase 2: Initialize Planning Team (4 Agents)

In [ ]:
# Import multi-agent system components
os.chdir(MULTI_AGENT_ROOT)

from agents.roles import Agent, AgentConfig, AgentTeam
from agents.router import AgentRouter, RoutingStrategy, Message

print("✅ Multi-agent system imported")

# Initialize Planning Team (4 agents)
planning_team_agents = {}
prompt_dir = Path(MULTI_AGENT_ROOT) / 'agents/prompts/planning_team'

# Define Planning Team configuration
planning_config = {
    'strategic_leader': {
        'name': 'Strategic Leader',
        'model': 'claude-opus-4-20250514',  # Using Opus for strategic decisions
        'provider': 'anthropic',
        'role': 'Lead strategic planning and make GO/PAUSE/PIVOT decisions',
        'prompt_file': '01_strategic_leader.md'
    },
    'empirical_validation': {
        'name': 'Empirical Validation Lead',
        'model': 'claude-sonnet-4-20250514',
        'provider': 'anthropic',
        'role': 'Validate statistical rigor and experimental methodology',
        'prompt_file': '02_empirical_validation_lead.md'
    },
    'critical_evaluator': {
        'name': 'Critical Evaluator',
        'model': 'gpt-4-turbo-2024-04-09',
        'provider': 'openai',
        'role': 'Challenge claims and identify methodological risks',
        'prompt_file': '03_critical_evaluator.md'
    },
    'research_advisor': {
        'name': 'Gemini Research Advisor',
        'model': 'gemini-2.0-flash-exp',
        'provider': 'google',
        'role': 'Provide literature context and alternative approaches',
        'prompt_file': '04_gemini_research_advisor.md'
    }
}

print("\n🤖 Initializing Planning Team:")
for agent_id, config in planning_config.items():
    agent_cfg = AgentConfig(
        name=config['name'],
        model=config['model'],
        provider=config['provider'],
        role=config['role'],
        prompt_file=config['prompt_file'],
        temperature=0.7,
        max_tokens=4000  # More tokens for strategic planning
    )
    planning_team_agents[agent_id] = Agent(agent_cfg, prompt_dir)
    print(f"   ✅ {config['name']} ({config['model']})")

# Create agent team and router
planning_team = AgentTeam(planning_team_agents)
planning_router = AgentRouter(planning_team)

print("\n✅ Planning Team initialized (4 agents)")

---
## Phase 3: Planning Team Reviews Results and Plans Next Cycle

In [ ]:
print("="*80)
print("📋 PLANNING TEAM REVIEW MEETING")
print("="*80)

# Create comprehensive review context for Planning Team
review_context = f"""# PLANNING TEAM REVIEW MEETING

## 📥 Executive Team Execution Results

{progress_text}

---

## 🎯 Your Task: Review Results and Plan Next Cycle

As the Planning Team, analyze the Executive Team's execution results and plan the next research cycle.

### 1. **Review Execution Results**

**Key Questions:**
- Which tasks completed successfully?
- Which tasks failed or were blocked? Why?
- What outputs and evidence were generated?
- Are the results statistically significant and reproducible?

**For this cycle:**
- Total tasks: {results_data.get('total_tasks', 'N/A')}
- Completed: {results_data.get('completed', 'N/A')}
- Failed: {results_data.get('failed', 'N/A')}

### 2. **Assess Progress Toward Week 1 GO/NO-GO Decision**

**Timeline:** Week 1 GO/NO-GO deadline is October 20, 2025
**Today:** {datetime.now().strftime('%Y-%m-%d')}

**Week 1 Validation Goals:**
- Diagnostic tools work on ≥3 external models (CLIP, ALIGN, Flamingo)
- Statistical evidence collected (p<0.05 threshold)
- CLIP diagnostic completed
- Results logged to MLflow

**Decision Criteria:**
- **GO (Field-wide paper):** ≥2 models show >80% attention imbalance → High-impact paper (72% acceptance)
- **PIVOT (Focused study):** Only V2 shows collapse → Focused case study (65% acceptance)
- **DIAGNOSTIC (Framework):** Tools work on all models → Methodology paper (85% acceptance - RECOMMENDED)

**Status Assessment:**
- Are we on track for Week 1 GO/NO-GO decision?
- Do we have sufficient statistical evidence?
- Have we tested enough models?
- What blockers or risks exist?

### 3. **Identify Next Cycle Priorities**

Based on execution results:

**If HIGH priority tasks completed successfully:**
- Move to next phase of validation
- Expand to additional models
- Begin paper writing

**If HIGH priority tasks failed:**
- Debug failures and re-run
- Adjust experimental approach
- Address blockers

**If blockers identified (e.g., ALIGN model access):**
- Evaluate alternative approaches
- Update timeline
- Adjust research scope

### 4. **Generate New `pending_actions.json` for Next Cycle**

Create a detailed action plan with:

**Required Format:**
```json
{{
  "meeting_id": "cvpr_planning_cycle2_{{timestamp}}",
  "generated_at": "{{ISO timestamp}}",
  "context": "Week 1 Cycle 2: [Brief description based on Cycle 1 results]",
  "decisions": [
    {{
      "priority": "HIGH",
      "action": "[Specific task based on execution results]",
      "owner": "ops_commander",
      "rationale": "[Why this task is needed based on Cycle 1 results]",
      "deadline": "{{ISO timestamp}}",
      "acceptance_criteria": [
        "[Specific, measurable criterion]"
      ],
      "dependencies": ["[task_ids if applicable]"],
      "evidence_paths": ["[File paths for verification]"]
    }}
  ],
  "acceptance_gates": {{
    "week1_validation": {{
      "min_models_tested": 3,
      "min_models_with_collapse": 2,
      "statistical_threshold": "p<0.05"
    }}
  }}
}}
```

**Task Prioritization Guidelines:**
- **HIGH:** Critical path items needed for Week 1 GO/NO-GO decision
- **MEDIUM:** Important but not blocking the decision
- **LOW:** Nice to have, can be deferred

**Typical next cycle tasks might include:**
- Run diagnostic on additional models (BLIP, Flamingo, LLaVA)
- Cross-model statistical comparison
- Update paper draft with new experimental results
- Address any blockers from previous cycle
- Prepare GO/NO-GO decision framework

---

## 🤖 Planning Team Roles

**Strategic Leader (Claude Opus 4):**
- Lead this meeting and synthesize team input
- Make final GO/PAUSE/PIVOT decisions
- Create the pending_actions.json structure
- Ensure timeline alignment

**Empirical Validation Lead (Claude Sonnet 4):**
- Assess statistical validity of execution results
- Verify experimental rigor
- Recommend next experiments based on evidence
- Validate that acceptance criteria are measurable

**Critical Evaluator (GPT-4):**
- Challenge claims (is the evidence sufficient?)
- Identify methodological risks
- Flag reproducibility issues
- Question optimistic assumptions

**Gemini Research Advisor (Gemini Flash):**
- Provide context from literature
- Suggest alternative approaches if blocked
- Assess feasibility of proposed next steps
- Connect findings to broader research landscape

---

## 📤 Required Output

Each agent should provide:

1. **Assessment of Cycle 1 Results:**
   - What worked?
   - What failed?
   - What did we learn?
   - Statistical validity?

2. **Progress Toward Week 1 GO/NO-GO:**
   - Are we on track?
   - What evidence do we have?
   - What gaps remain?
   - Risk assessment?

3. **Recommended Next Cycle Tasks:**
   - What should be HIGH priority?
   - What can wait?
   - What blockers need addressing?
   - Timeline implications?

**Strategic Leader will then synthesize all input and generate the final `pending_actions.json`.**

---

## 🚨 Critical Thinking Required

**Don't just continue blindly:**
- If results show attention collapse, verify statistical significance
- If results show no collapse, investigate why (is it V2-specific?)
- If blockers emerged (like ALIGN access), evaluate alternatives
- If we're behind schedule, what can be cut or parallelized?
- If we're ahead, what stretch goals can strengthen the paper?

**Evidence-based decisions:**
- Every priority must be justified by execution results
- Every new task must reference what previous cycle showed
- Every deadline must account for remaining time to Nov 6 abstract deadline

---

**Begin your review now. Provide your analysis and recommendations.**
"""

print("\n📝 Sending review context to Planning Team...\n")

# Collect responses from all Planning Team agents
planning_responses = {}

message = Message(
    sender="cycle_coordinator",
    recipients=list(planning_team_agents.keys()),
    content=review_context,
    message_type="planning_review"
)

print("🤖 Planning Team agents reviewing results...\n")

responses = planning_router.route_message(message)

# Display each agent's response
for agent_name, response in responses.items():
    planning_responses[agent_name] = response
    print("="*80)
    print(f"📝 {planning_config[agent_name]['name']} Response")
    print("="*80)
    print(response)
    print("\n")

print("="*80)
print("✅ All Planning Team agents have provided their analysis")
print("="*80)

---
## Phase 4: Strategic Leader Generates Next Cycle Tasks

In [ ]:
print("="*80)
print("🎯 STRATEGIC LEADER: GENERATE NEXT CYCLE TASKS")
print("="*80)

# Strategic Leader synthesizes all input and creates pending_actions.json
synthesis_prompt = f"""# STRATEGIC LEADER: SYNTHESIZE AND GENERATE NEXT CYCLE PLAN

## Team Input Received:

### Empirical Validation Lead:
{planning_responses.get('empirical_validation', 'No response')}

### Critical Evaluator:
{planning_responses.get('critical_evaluator', 'No response')}

### Gemini Research Advisor:
{planning_responses.get('research_advisor', 'No response')}

---

## Your Task:

As Strategic Leader, synthesize all team input and create the final `pending_actions.json` for Cycle 2.

**Requirements:**

1. **Incorporate team feedback:**
   - Statistical rigor concerns from Empirical Lead
   - Risk flags from Critical Evaluator
   - Alternative approaches from Research Advisor

2. **Create actionable tasks:**
   - Based on Cycle 1 execution results
   - Prioritized by impact on Week 1 GO/NO-GO decision
   - With clear acceptance criteria
   - Realistic deadlines

3. **Format as valid JSON:**
   - Must be parseable JSON (no markdown formatting inside JSON)
   - Include meeting_id, generated_at, context, decisions array
   - Each decision must have: priority, action, owner, rationale, deadline, acceptance_criteria, evidence_paths

**Output ONLY the JSON structure, no additional commentary.**

**Example format:**
```json
{{
  "meeting_id": "cvpr_planning_cycle2_{datetime.now().strftime('%Y%m%d_%H%M%S')}",
  "generated_at": "{datetime.now().isoformat()}",
  "context": "Week 1 Cycle 2: [Your strategic summary based on team input]",
  "decisions": [
    {{
      "priority": "HIGH",
      "action": "[Task based on Cycle 1 results]",
      "owner": "ops_commander",
      "rationale": "[Why this task, referencing Cycle 1 findings]",
      "deadline": "{(datetime.now() + timedelta(days=2)).isoformat()}",
      "acceptance_criteria": [
        "[Criterion 1]",
        "[Criterion 2]"
      ],
      "dependencies": [],
      "evidence_paths": ["[Path to verify completion]"]
    }}
  ],
  "acceptance_gates": {{
    "week1_validation": {{
      "min_models_tested": 3,
      "min_models_with_collapse": 2,
      "statistical_threshold": "p<0.05"
    }}
  }}
}}
```

Generate the complete pending_actions.json now.
"""

from datetime import timedelta

# Get Strategic Leader to generate final plan
strategic_leader = planning_team_agents['strategic_leader']

print("\n🤖 Strategic Leader generating next cycle plan...\n")

next_cycle_json = strategic_leader.respond(synthesis_prompt)

print("="*80)
print("📋 NEXT CYCLE PENDING ACTIONS")
print("="*80)
print(next_cycle_json)
print("\n" + "="*80)

---
## Phase 5: Save New pending_actions.json to Google Drive

In [ ]:
import json
import re

print("="*80)
print("💾 SAVING NEXT CYCLE PENDING ACTIONS")
print("="*80)

# Extract JSON from response (in case it's wrapped in markdown)
json_match = re.search(r'```json\s*({.*?})\s*```', next_cycle_json, re.DOTALL)
if json_match:
    json_str = json_match.group(1)
else:
    # Try to find raw JSON
    json_match = re.search(r'({\s*"meeting_id".*})', next_cycle_json, re.DOTALL)
    if json_match:
        json_str = json_match.group(1)
    else:
        json_str = next_cycle_json

try:
    # Parse JSON to validate
    pending_actions = json.loads(json_str)
    
    # Save to handoff directory
    output_file = Path(MULTI_AGENT_ROOT) / 'reports/handoff/pending_actions.json'
    
    with open(output_file, 'w') as f:
        json.dump(pending_actions, f, indent=2)
    
    print(f"\n✅ New pending_actions.json saved")
    print(f"   File: {output_file}")
    print(f"   Meeting ID: {pending_actions.get('meeting_id', 'N/A')}")
    print(f"   Context: {pending_actions.get('context', 'N/A')}")
    print(f"   Total tasks: {len(pending_actions.get('decisions', []))}")
    
    # Count by priority
    decisions = pending_actions.get('decisions', [])
    high = len([d for d in decisions if d.get('priority') == 'HIGH'])
    medium = len([d for d in decisions if d.get('priority') == 'MEDIUM'])
    low = len([d for d in decisions if d.get('priority') == 'LOW'])
    
    print(f"\n📊 Task Breakdown:")
    print(f"   ⭐ HIGH: {high}")
    print(f"   🟠 MEDIUM: {medium}")
    print(f"   🔵 LOW: {low}")
    
    # Also save timestamped backup
    backup_dir = Path(MULTI_AGENT_ROOT) / 'reports/planning/pending_actions_history'
    backup_dir.mkdir(parents=True, exist_ok=True)
    
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    backup_file = backup_dir / f'pending_actions_{timestamp}.json'
    
    with open(backup_file, 'w') as f:
        json.dump(pending_actions, f, indent=2)
    
    print(f"\n✅ Backup saved: {backup_file.name}")
    
except json.JSONDecodeError as e:
    print(f"\n❌ Error: Could not parse JSON from Strategic Leader response")
    print(f"   Error: {e}")
    print(f"\n📄 Raw response:")
    print(next_cycle_json)
    print(f"\n⚠️ You may need to manually create pending_actions.json")

print("\n" + "="*80)

---
## Summary: Planning Team Review Complete

In [ ]:
print("="*80)
print("✅ PLANNING TEAM REVIEW MEETING COMPLETE")
print("="*80)

print("\n📋 Meeting Summary:")
print(f"   ✅ Reviewed Cycle 1 execution results")
print(f"   ✅ All 4 Planning Team agents provided analysis")
print(f"   ✅ Strategic Leader synthesized input")
print(f"   ✅ New pending_actions.json generated for Cycle 2")

print("\n📂 Output Files:")
print(f"   📄 reports/handoff/pending_actions.json (new cycle tasks)")
print(f"   📄 reports/planning/pending_actions_history/pending_actions_{timestamp}.json (backup)")

print("\n🔄 Next Steps:")
print(f"   1. Review the new pending_actions.json")
print(f"   2. Run Executive Team execution cycle (Colab notebook)")
print(f"   3. Execute Cycle 2 tasks")
print(f"   4. Return here for next Planning Team review")

print("\n" + "="*80)
print("🚀 READY FOR CYCLE 2 EXECUTION")
print("="*80)